# Продвинутое программирование на языке Python
## Семинар 1. Принципы ООП

На этом занятии постараемся разобрать логику `.__init__()` -> `.fit()` -> `.transform()` (`.predict()`). Это пригодится нам при работе с `sklearn`.

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

In [2]:
X = np.random.uniform(-10, 10, (100, 2))
y = np.random.normal(0, 1, 100)

X_new = np.random.uniform(-10, 10, (50, 2))

In [3]:
lr = LinearRegression()  # .__init__()
# Y = b0*1 + b1*X1 + b2*X2 + ... + bn*Xn

In [4]:
dir(lr)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_set_intercept',
 '_validate_data',
 '_validate_params',
 'copy_X',
 'fit',
 'fit_intercept',
 'get_metadata_routing',
 'get_params',
 'n_jobs',
 'positive',
 'predict',
 'score',
 'set_fit_request',
 's

In [5]:
lr.fit(X, y)

LinearRegression()

In [6]:
dir(lr)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_set_intercept',
 '_validate_data',
 '_validate_params',
 'coef_',
 'copy_X',
 'fit',
 'fit_intercept',
 'get_metadata_routing',
 'get_params',
 'intercept_',
 'n_features_in_',
 'n_jobs',
 'positive',
 

In [7]:
_ = lr.predict(X_new)

В процессе анализа данных нам часто нужно будет трансформировать их каким-либо образом. Рассмотрим две главных трансформации: стадартизацию и нормализацию.

![st&norm](https://i.stack.imgur.com/XmyWR.png)

Наша задача - реализовать класс `Transformer`.

При этом:
- для инициализации класса необходим вид трансформации;
- на вход может подаваться последовательность из целых / нецелых чисел (возможные виды: `list`, `tuple`, `np.ndarray`, `pd.DataFrame`). Последовательности (если это не `np.ndarray` или `pd.DataFrame`) должны иметь степень вложенности 2 (список списков);
- параметры для трансформации должны сохраняться в качестве атрибутов объекта (для каждой переменной);
- должна поддерживаться логика `fit`-`transform`.

In [40]:
class Transformer:
    _valid_ttypes = ['norm', 'std']
    _valid_structures = [list, tuple, np.ndarray, pd.DataFrame]
    _valid_dtypes = [int, float]
    
    def __init__(self, ttype='norm'):
        if ttype not in self._valid_ttypes:
            raise ValueError(f'invalid ttype {ttype}, use only authorized ttypes: {self._valid_ttypes}!')
            
        self.ttype = ttype

    def _validate_data(self, data):
        # check data structure
        structure = type(data)
        if structure not in self._valid_structures:
            raise ValueError(f'invalid structure {structure}, use only authorized structures: {self._valid_structures}!')
        
        # <data> -> np.ndarray
        if structure in [list, tuple]:
            data = np.array(data)
        elif structure == pd.DataFrame:
            data = data.values
        
        # check dtype
        if data.dtype not in self._valid_dtypes:
            raise ValueError(f'invalid dtype {data.dtype}, use only authorized dtypes: {self._valid_dtypes}!')

        # shape
        if data.ndim != 2:
            raise ValueError(f'invalid shape {data.ndim}, must be 2!')

        return data

    def fit(self, data):
        # validate data
        data_valid = self._validate_data(data)
        
        # compute params
        if self.ttype == 'norm':
            mins = data_valid.min(axis=0)
            maxs = data_valid.max(axis=0)
            
            params = {
                'min': mins,
                'max': maxs
            }
        else:
            means = data_valid.mean(axis=0)
            stds = data_valid.std(axis=0)
            
            params = {
                'mean': means,
                'std': stds
            }
            
        # save params
        self.params = params

    def transform(self, data) -> pd.DataFrame:
        # check if fit is applied
        if not hasattr(self, 'params'):
            raise AttributeError('you cannot apply transform before fit!')
        
        # validate data
        data_valid = self._validate_data(data)

        # apply params to data
        if self.ttype == 'norm':
            data_transformed = (data_valid - self.params['min']) / (self.params['max'] - self.params['min'])
            
        else:
            data_transformed = (data_valid - self.params['mean']) / self.params['std']

        return data_transformed

In [45]:
tr = Transformer(ttype='std')
tr.fit(X)

X_std = tr.transform(X)

In [46]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X)

X_std = sc.transform(X)

In [24]:
tr._Transformer__top_secret_attribute

'top_secret'

In [21]:
tr = Transformer()

In [19]:
tr._valid_ttypes

['norm', 'std']

In [ ]:
tr.

In [ ]:
tr.ttype

In [23]:
dir(tr)

['_Transformer__top_secret_attribute',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_valid_ttypes',
 'fit',
 'transform',
 'ttype']

Теперь давайте попробуем немного изменить логику и реализовать родительский класс Transformer и два наследуемых: StandardScaler и NormScaler.

In [47]:
class Transformer:
    _valid_structures = [list, tuple, np.ndarray, pd.DataFrame]
    _valid_dtypes = [int, float]
    
    def __init__(self):
        pass

    def _validate_data(self, data):
        # check data structure
        structure = type(data)
        if structure not in self._valid_structures:
            raise ValueError(f'invalid structure {structure}, use only authorized structures: {self._valid_structures}!')
        
        # <data> -> np.ndarray
        if structure in [list, tuple]:
            data = np.array(data)
        elif structure == pd.DataFrame:
            data = data.values
        
        # check dtype
        if data.dtype not in self._valid_dtypes:
            raise ValueError(f'invalid dtype {data.dtype}, use only authorized dtypes: {self._valid_dtypes}!')

        # shape
        if data.ndim != 2:
            raise ValueError(f'invalid shape {data.ndim}, must be 2!')

        return data

    def _get_params(self, data):  # must be specified in child class
        pass

    def _apply_params(self, data):  # must be specified in child class
        pass

    def fit(self, data):
        # validate data
        data_valid = self._validate_data(data)
        
        # compute params
        params = self._get_params(data_valid)
            
        # save params
        self.params = params

    def transform(self, data) -> pd.DataFrame:
        # check if fit is applied
        if not hasattr(self, 'params'):
            raise AttributeError('you cannot apply transform before fit!')
        
        # validate data
        data_valid = self._validate_data(data)

        # apply params to data
        data_transformed = self._apply_params(data_valid)

        return data_transformed

In [48]:
class StandardScaler(Transformer):
    def _get_params(self, data):
        means = data.mean(axis=0)
        stds = data.std(axis=0)
        
        params = {
            'mean': means,
            'std': stds
        }

        return params

    def _apply_params(self, data):
        data_transformed = (data - self.params['mean']) / self.params['std']

        return data_transformed

